In [12]:
### 모듈 및 데이터 로딩
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
file = '../0. Data/2. output/3. knn_imp_Train_data.csv'
data = pd.read_csv(file)

In [13]:
### 모델 생성을 위한 Train 데이터 증강 by KNN Imputer


# scale_pv가 2 이하인 데이터를 NaN으로 대체
data_imputed = data.copy()
data_imputed.loc[data_imputed['scale_pv'] <= 2, 'scale_pv'] = np.nan


# KNN Imputer 적용
imputer = KNNImputer(n_neighbors=5)
data_imputed[['scale_pv']] = imputer.fit_transform(data_imputed[['scale_pv']])


# 예측된 scale_pv 값을 2초과 4미만 범위로 조정
data_imputed['scale_pv'] = np.clip(data_imputed['scale_pv'], 2.01, 3.99)

KeyboardInterrupt: 

In [ ]:
### 모델링 및 Train 평가


# 중복값 제거
data_imputed.drop_duplicates(inplace=True)


# 피처와 타겟 분리
X = data_imputed.drop(columns=['scale_pv'])
y = data_imputed['scale_pv']


# 학습 데이터와 검증 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)


# 타겟 스케일링
target_scaler = StandardScaler()
y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_valid_scaled = target_scaler.transform(y_valid.values.reshape(-1, 1))


### Grid Search를 사용한 최적의 파라미터 찾기

# Linear Regression - GridSearchCV
lr_param_grid = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'n_jobs': [None, -1],
    'positive': [True, False]
}
lr_grid_search = GridSearchCV(LinearRegression(), lr_param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
lr_grid_search.fit(X_train_scaled, y_train_scaled.ravel())

print(f"Best parameters for Linear Regression: {lr_grid_search.best_params_}")
print(f"Best CV score (MAE) for Linear Regression: {-lr_grid_search.best_score_}")

# Random Forest - GridSearchCV
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}
rf_grid_search = GridSearchCV(RandomForestRegressor(random_state=42), rf_param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
rf_grid_search.fit(X_train_scaled, y_train_scaled.ravel())

print(f"Best parameters for Random Forest: {rf_grid_search.best_params_}")
print(f"Best CV score (MAE) for Random Forest: {-rf_grid_search.best_score_}")

# LightGBM - GridSearchCV
lgb_param_grid = {
    'num_leaves': [31, 50, 100],
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [50, 100, 200],
    'max_depth': [-1, 10, 20, 30]
}
lgb_grid_search = GridSearchCV(lgb.LGBMRegressor(random_state=42), lgb_param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
lgb_grid_search.fit(X_train_scaled, y_train_scaled.ravel())

print(f"Best parameters for LightGBM: {lgb_grid_search.best_params_}")
print(f"Best CV score (MAE) for LightGBM: {-lgb_grid_search.best_score_}")


# 최적의 모델로 학습 및 평가

# 최적의 파라미터로 모델 재학습
lr_best_model = lr_grid_search.best_estimator_
rf_best_model = rf_grid_search.best_estimator_
lgb_best_model = lgb_grid_search.best_estimator_


# 모델 학습 및 평가 함수
def train_and_evaluate_model(model, X_train, X_valid, y_train_scaled, y_valid_scaled, target_scaler):
    model.fit(X_train, y_train_scaled.ravel())
    y_train_pred_scaled = model.predict(X_train)
    y_valid_pred_scaled = model.predict(X_valid)
    
    # 역스케일링
    y_train_pred = target_scaler.inverse_transform(y_train_pred_scaled.reshape(-1, 1))
    y_valid_pred = target_scaler.inverse_transform(y_valid_pred_scaled.reshape(-1, 1))
    y_train_original = target_scaler.inverse_transform(y_train_scaled)
    y_valid_original = target_scaler.inverse_transform(y_valid_scaled)
    
    train_mae = mean_absolute_error(y_train_original, y_train_pred)
    valid_mae = mean_absolute_error(y_valid_original, y_valid_pred)
    train_mape = mean_absolute_percentage_error(y_train_original, y_train_pred)
    valid_mape = mean_absolute_percentage_error(y_valid_original, y_valid_pred)
    
    return train_mae, valid_mae, train_mape, valid_mape, y_train_pred, y_valid_pred


# 모델 학습 및 평가

# Multiple Regression
lr_train_mae, lr_valid_mae, lr_train_mape, lr_valid_mape, lr_y_train_pred, lr_y_valid_pred = train_and_evaluate_model(lr_best_model, X_train_scaled, X_valid_scaled, y_train_scaled, y_valid_scaled, target_scaler)

# Random Forest
rf_train_mae, rf_valid_mae, rf_train_mape, rf_valid_mape, rf_y_train_pred, rf_y_valid_pred = train_and_evaluate_model(rf_best_model, X_train_scaled, X_valid_scaled, y_train_scaled, y_valid_scaled, target_scaler)

# LightGBM
lgb_train_mae, lgb_valid_mae, lgb_train_mape, lgb_valid_mape, lgb_y_train_pred, lgb_y_valid_pred = train_and_evaluate_model(lgb_best_model, X_train_scaled, X_valid_scaled, y_train_scaled, y_valid_scaled, target_scaler)


# 결과 출력
print(f"Linear Regression - Train MAE: {lr_train_mae}, Train MAPE: {lr_train_mape*100}")
print(f"Linear Regression - Valid MAE: {lr_valid_mae}, Valid MAPE: {lr_valid_mape*100}")
print()
print(f"Random Forest - Train MAE: {rf_train_mae}, Train MAPE: {rf_train_mape*100}")
print(f"Random Forest - Valid MAE: {rf_valid_mae}, Valid MAPE: {rf_valid_mape*100}")
print()
print(f"LightGBM - Train MAE: {lgb_train_mae}, Train MAPE: {lgb_train_mape*100}")
print(f"LightGBM - Valid MAE: {lgb_valid_mae}, Valid MAPE: {lgb_valid_mape*100}")

Best parameters for Linear Regression: {'copy_X': True, 'fit_intercept': False, 'n_jobs': None, 'positive': True}
Best CV score (MAE) for Linear Regression: 0.37341008169551293
Best parameters for Random Forest: {'bootstrap': True, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best CV score (MAE) for Random Forest: 0.37507885667090607
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005243 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 227
[LightGBM] [Info] Number of data points in the train set: 107248, number of used features: 4
[LightGBM] [Info] Start training from score 0.000000
Best parameters for LightGBM: {'learning_rate': 0.01, 'max_depth': -1, 'n_estimators': 50, 'num_leaves': 31}
Best CV score (MAE) for LightGBM: 0.353598953591645
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002502 sec

In [16]:
# import joblib

# # 모델 및 스케일러 저장
# scaler_data = {
#     'scaler': scaler,
#     'feature_names': X.columns.tolist(),
#     'target_scaler': target_scaler
# }
# joblib.dump(rf_best_model, '../2. Modeling/model/kIG_rf_model.pkl')
# joblib.dump(scaler_data, '../2. Modeling/model/kIG_rf_scaler.pkl')

# print("Model saved successfully!")

Model saved successfully!


In [ ]:
### Test 평가

# Test_data 로딩
test_file = '../0. Data/2. output/0. Test_data.csv'
test_data = pd.read_csv(test_file)

# 피처와 타겟 분리
X_test_final = test_data.drop(columns=['scale_pv'])
y_test_final = test_data['scale_pv']

# 테스트 데이터 스케일링
X_test_final_scaled = scaler.transform(X_test_final)
y_test_final_scaled = target_scaler.transform(y_test_final.values.reshape(-1, 1))

# 최종 테스트 데이터 예측 및 평가
def final_evaluate_model(model, X_test_scaled, y_test_scaled, target_scaler):
    y_test_pred_scaled = model.predict(X_test_scaled)
    
    # 역스케일링
    y_test_pred = target_scaler.inverse_transform(y_test_pred_scaled.reshape(-1, 1))
    y_test_original = target_scaler.inverse_transform(y_test_scaled)
    
    test_mae = mean_absolute_error(y_test_original, y_test_pred)
    test_mape = mean_absolute_percentage_error(y_test_original, y_test_pred)
    
    return test_mae, test_mape, y_test_pred

# 최종 평가 결과
lr_test_mae_final, lr_test_mape_final, lr_y_test_pred_final = final_evaluate_model(lr_best_model, X_test_final_scaled, y_test_final_scaled, target_scaler)
rf_test_mae_final, rf_test_mape_final, rf_y_test_pred_final = final_evaluate_model(rf_best_model, X_test_final_scaled, y_test_final_scaled, target_scaler)
lgb_test_mae_final, lgb_test_mape_final, lgb_y_test_pred_final = final_evaluate_model(lgb_best_model, X_test_final_scaled, y_test_final_scaled, target_scaler)

print(f"Final Test - Linear Regression MAE: {lr_test_mae_final}, MAPE: {lr_test_mape_final*100}")
print(f"Final Test - Random Forest MAE: {rf_test_mae_final}, MAPE: {rf_test_mape_final*100}")
print(f"Final Test - LightGBM MAE: {lgb_test_mae_final}, MAPE: {lgb_test_mape_final*100}")

Final Test - Linear Regression MAE: 0.023359419107954284, MAPE: 0.7632538471508572
Final Test - Random Forest MAE: 0.023537679168576266, MAPE: 0.7693245496258292
Final Test - LightGBM MAE: 0.023668570021191427, MAPE: 0.7732892630046533
